In [45]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf
from keras.layers import Dense
from keras.models import Sequential
from keras.optimizers import Adam

class TFModel:
    def _create_layer(self, input_layer, layer_weights, layer_bias, activation_func):
        layer = tf.add(tf.matmul(input_layer, layer_weights), layer_bias)
        return activation_func(layer)
    
    def __init__(self, num_inputs, num_outputs, hidden_layer_sizes):
        self.num_inputs = num_inputs
        self.num_outputs = num_outputs
        
        layer_sizes = [self.num_inputs] + hidden_layer_sizes + [self.num_outputs]
        num_layers = len(layer_sizes)
        
        weights = list(
            map(
                lambda input_size, output_size: tf.Variable(
                    tf.random_normal(shape=[input_size, output_size])
                ),
                layer_sizes,
                layer_sizes[1:],
            )
        )
        
        biases = list(
            map(
                lambda layer_size: tf.Variable(tf.random_normal(shape=[layer_size])),
                layer_sizes[1:],
            )
        )
        
        activation_funcs = list(
            map(
                lambda layer_index: tf.identity
                if layer_index == num_layers - 2
                else tf.nn.relu,
                range(num_layers - 1),
            )
        )
        
        self._input = tf.placeholder(shape=[None, self.num_inputs], dtype=tf.float32)
        layer = self._input
        for layer_weights, layer_bias, activation_func in zip(
            weights, biases, activation_funcs
        ):
            layer = self._create_layer(
                layer, layer_weights, layer_bias, activation_func
            )
        
        self._output = layer
        
        self._target_output = tf.placeholder(
            shape=[None, self.num_outputs], dtype=tf.float32
        )
        loss = tf.losses.mean_squared_error(self._target_output, self._output)
        self._optimizer = tf.train.AdamOptimizer().minimize(loss)
        
        self.init = tf.global_variables_initializer()
    
    def predict(self, sess, input):
        return sess.run(self._output, feed_dict={self._input: input})

    def predict_batch(self, sess, inputs):
        return sess.run(self._output, feed_dict={self._input: inputs})

    def train_batch(self, sess, inputs, outputs):
        sess.run(
            self._optimizer,
            feed_dict={self._input: inputs, self._target_output: outputs},
        )
        
def compile_keras_model(input, output, number_of_nodes_in_hidden_layer=3, number_of_hidden_layers=2, hidden_layer_activation='relu', output_layer_activation='linear', loss='mse'):
    model = Sequential()
    model.add(Dense(number_of_nodes_in_hidden_layer, activation=hidden_layer_activation, input_shape=(len(input.columns),)))
    for _ in range(number_of_hidden_layers - 1):  
        model.add(Dense(number_of_nodes_in_hidden_layer, activation=hidden_layer_activation))
    model.add(Dense(1, activation=output_layer_activation))
    
    adam = Adam()
    model.compile(loss=loss, optimizer=adam, metrics=['acc'])
    
    return model

In [50]:
data = pd.read_csv("dataset/yes_date_set.csv")
X = data.drop(['trip_count', 'date', 'Holiday_None'], axis=1)
print(X)
print(y.head())
y = data[['trip_count']]
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=42)

      max_temperature_f  mean_temperature_f  min_temperature_f  \
0                  74.0                68.0               61.0   
1                  78.0                69.0               60.0   
2                  71.0                64.0               57.0   
3                  74.0                66.0               58.0   
4                  75.0                69.0               62.0   
5                  73.0                67.0               60.0   
6                  74.0                68.0               61.0   
7                  72.0                66.0               60.0   
8                  85.0                71.0               56.0   
9                  88.0                73.0               58.0   
10                 74.0                65.0               56.0   
11                 76.0                66.0               55.0   
12                 74.0                66.0               57.0   
13                 74.0                68.0               62.0   
14        

In [51]:
model = TFModel(len(X.columns), 1, [3, 3])
with tf.Session() as sess:
    sess.run(model.init)
    
    model.train_batch(sess, X, y)
    predictions = model.predict_batch(sess, X)
    print(predictions)

[[nan]
 [nan]
 [nan]
 ...
 [nan]
 [nan]
 [nan]]


In [52]:
model = compile_keras_model(X, y)
model.fit(X_train, y_train, epochs=150, validation_data=(X_test, y_test), verbose=1)

Train on 3846 samples, validate on 1649 samples
Epoch 1/150
3846/3846 [==============================] - 1s 157us/step - loss: nan - acc: 2.6001e-04 - val_loss: nan - val_acc: 0.0000e+00
Epoch 2/150
3846/3846 [==============================] - 0s 62us/step - loss: nan - acc: 0.0000e+00 - val_loss: nan - val_acc: 0.0000e+00
Epoch 3/150
3846/3846 [==============================] - 0s 73us/step - loss: nan - acc: 0.0000e+00 - val_loss: nan - val_acc: 0.0000e+00
Epoch 4/150
3846/3846 [==============================] - 0s 72us/step - loss: nan - acc: 0.0000e+00 - val_loss: nan - val_acc: 0.0000e+00
Epoch 5/150
3846/3846 [==============================] - 0s 70us/step - loss: nan - acc: 0.0000e+00 - val_loss: nan - val_acc: 0.0000e+00
Epoch 6/150
3846/3846 [==============================] - 0s 70us/step - loss: nan - acc: 0.0000e+00 - val_loss: nan - val_acc: 0.0000e+00
Epoch 7/150
3846/3846 [==============================] - 0s 74us/step - loss: nan - acc: 0.0000e+00 - val_loss: nan - val_a

3846/3846 [==============================] - 0s 84us/step - loss: nan - acc: 0.0000e+00 - val_loss: nan - val_acc: 0.0000e+00
Epoch 60/150
3846/3846 [==============================] - 0s 80us/step - loss: nan - acc: 0.0000e+00 - val_loss: nan - val_acc: 0.0000e+00
Epoch 61/150
3846/3846 [==============================] - 0s 56us/step - loss: nan - acc: 0.0000e+00 - val_loss: nan - val_acc: 0.0000e+00
Epoch 62/150
3846/3846 [==============================] - 0s 59us/step - loss: nan - acc: 0.0000e+00 - val_loss: nan - val_acc: 0.0000e+00
Epoch 63/150
3846/3846 [==============================] - 0s 57us/step - loss: nan - acc: 0.0000e+00 - val_loss: nan - val_acc: 0.0000e+00
Epoch 64/150
3846/3846 [==============================] - 0s 102us/step - loss: nan - acc: 0.0000e+00 - val_loss: nan - val_acc: 0.0000e+00
Epoch 65/150
3846/3846 [==============================] - 0s 65us/step - loss: nan - acc: 0.0000e+00 - val_loss: nan - val_acc: 0.0000e+00
Epoch 66/150
3846/3846 [===============

Epoch 118/150
3846/3846 [==============================] - 0s 63us/step - loss: nan - acc: 0.0000e+00 - val_loss: nan - val_acc: 0.0000e+00
Epoch 119/150
3846/3846 [==============================] - 0s 62us/step - loss: nan - acc: 0.0000e+00 - val_loss: nan - val_acc: 0.0000e+00
Epoch 120/150
3846/3846 [==============================] - 0s 63us/step - loss: nan - acc: 0.0000e+00 - val_loss: nan - val_acc: 0.0000e+00
Epoch 121/150
3846/3846 [==============================] - 0s 79us/step - loss: nan - acc: 0.0000e+00 - val_loss: nan - val_acc: 0.0000e+00
Epoch 122/150
3846/3846 [==============================] - 0s 67us/step - loss: nan - acc: 0.0000e+00 - val_loss: nan - val_acc: 0.0000e+00
Epoch 123/150
3846/3846 [==============================] - 0s 61us/step - loss: nan - acc: 0.0000e+00 - val_loss: nan - val_acc: 0.0000e+00
Epoch 124/150
3846/3846 [==============================] - 0s 65us/step - loss: nan - acc: 0.0000e+00 - val_loss: nan - val_acc: 0.0000e+00
Epoch 125/150
3846/3